In [1]:
import tensorflow as tf

In [110]:
def AllTriplesSet(rng):
	"""Returns all possible triples of integers between zero and natom.

	Args:
		rng: a 1D integer tensor to be triply outer product'd
	Returns:
		A Nmol X natom X natom X natom X 4 tensor of all triples.
	"""
	natom = tf.shape(rng)[1]
	nmol = tf.shape(rng)[0]
	v1 = tf.tile(tf.reshape(rng,[nmol,natom,1]),[1,1,natom])
	v2 = tf.tile(tf.reshape(rng,[nmol,1,natom]),[1,natom,1])
	v3 = tf.transpose(tf.stack([v1,v2],1),perm=[0,2,3,1])
	# V3 is now all pairs (nat x nat x 2). now do the same with another to make nat X 3
	v4 = tf.tile(tf.reshape(v3,[nmol,natom,natom,1,2]),[1,1,1,natom,1])
	v5 = tf.tile(tf.reshape(rng,[nmol,1,1,natom,1]),[1,natom,natom,1,1])
	v6 = tf.concat([v4,v5], axis = 4) # All triples in the range.
	v7 = tf.tile(tf.reshape(tf.range(nmol),[nmol,1,1,1,1]),[1,natom,natom,natom,1])
	v8 = tf.concat([v7,v6], axis = -1)
	return v8

def ZouterSet(Z):
	"""
	Returns the outer product of atomic numbers for all molecules.

	Args:
		Z: nMol X MaxNAtom X 1 Z tensor
	Returns
		Z1Z2: nMol X MaxNAtom X MaxNAtom X 2 Z1Z2 tensor.
	"""
	zshp = tf.shape(Z)
	Zs = tf.reshape(Z,[zshp[0],zshp[1],1])
	z1 = tf.tile(Zs, [1,1,zshp[1]])
	z2 = tf.transpose(z1,perm=[0,2,1])
	return tf.transpose(tf.stack([z1,z2],axis=1),perm=[0,2,3,1])

def DifferenceVectorsSet(r_,prec = tf.float64):
	"""
	Given a nmol X maxnatom X 3 tensor of coordinates this
	returns a nmol X maxnatom X maxnatom X 3 tensor of Rij
	"""
	natom = tf.shape(r_)[1]
	nmol = tf.shape(r_)[0]
	#ri = tf.tile(tf.reshape(r_,[nmol,1,natom,3]),[1,natom,1,1])
	ri = tf.tile(tf.reshape(tf.cast(r_,prec),[nmol,1,natom*3]),[1,natom,1])
	ri = tf.reshape(ri, [nmol, natom, natom, 3])
	rj = tf.transpose(ri,perm=(0,2,1,3))
	return (ri-rj)

In [179]:
def TFSymASet(R, Zs, eleps_, SFPs_, R_cut, prec=tf.float64):
	"""
	A tensorflow implementation of the angular AN1 symmetry function for a single input molecule.
	Here j,k are all other atoms, but implicitly the output
	is separated across elements as well. eleps_ is a list of element pairs
	G = 2**(1-zeta) \sum_{j,k \neq i} (Angular triple) (radial triple) f_c(R_{ij}) f_c(R_{ik})
	a-la MolEmb.cpp. Also depends on PARAMS for zeta, eta, theta_s r_s
	This version improves on the previous by avoiding some
	heavy tiles.

	Args:
	    R: a nmol X maxnatom X 3 tensor of coordinates.
	    Zs : nmol X maxnatom X 1 tensor of atomic numbers.
	    eleps_: a nelepairs X 2 tensor of element pairs present in the data.
	    SFP: A symmetry function parameter tensor having the number of elements
	    as the SF output. 4 X nzeta X neta X thetas X nRs. For example, SFPs_[0,0,0,0,0]
	    is the first zeta parameter. SFPs_[3,0,0,0,1] is the second R parameter.
	    R_cut: Radial Cutoff
	    prec: a precision. 
	Returns:
	    Digested Mol. In the shape nmol X maxnatom X nelepairs X nZeta X nEta X nThetas X nRs
	"""
	inp_shp = tf.shape(R)
	nmol = inp_shp[0]
	natom = inp_shp[1]
	natom2 = natom*natom
	natom3 = natom*natom2
	nelep = tf.shape(eleps_)[0]
	pshape = tf.shape(SFPs_)
	nzeta = pshape[1]
	neta = pshape[2]
	ntheta = pshape[3]
	nr = pshape[4]
	nsym = nzeta*neta*ntheta*nr
	infinitesimal = 0.000000000000000000000000001

	# atom triples.
	ats = AllTriplesSet(tf.tile(tf.reshape(tf.range(natom),[1,natom]),[nmol,1]))
	# before performing any computation reduce this to desired pairs.
	# Construct the angle triples acos(<Rij,Rik>/|Rij||Rik|) and mask them onto the correct output
	# Get Rij, Rik...
	Rm_inds = tf.slice(ats,[0,0,0,0,0],[nmol,natom,natom,natom,1])
	Ri_inds = tf.slice(ats,[0,0,0,0,1],[nmol,natom,natom,natom,1])
	Rj_inds = tf.slice(ats,[0,0,0,0,2],[nmol,natom,natom,natom,1])
	Rk_inds = tf.slice(ats,[0,0,0,0,3],[nmol,natom,natom,natom,1])
	Rjk_inds = tf.reshape(tf.concat([Rm_inds,Rj_inds,Rk_inds],axis=4),[nmol,natom3,3])
	Z1Z2 = ZouterSet(Zs)
	ZPairs = tf.gather_nd(Z1Z2,Rjk_inds) # should have shape nmol X natom3 X 2
	ElemReduceMask = tf.reduce_all(tf.equal(tf.reshape(ZPairs,[nmol,natom3,1,2]),tf.reshape(eleps_,[1,1,nelep,2])),axis=-1) # nmol X natom3 X nelep
	# Zero out the diagonal contributions (i==j or i==k)
	IdentMask = tf.tile(tf.reshape(tf.logical_and(tf.not_equal(Ri_inds,Rj_inds),tf.not_equal(Ri_inds,Rk_inds)),[nmol,natom3,1]),[1,1,nelep])
	Mask = tf.logical_and(ElemReduceMask,IdentMask) # nmol X natom3 X nelep
	# Mask is true if atoms ijk => pair_l and many triples are unused.  
	# So we create a final index tensor, which is only nonzero m,ijk,l
	pinds = tf.range(nelep)
	ats = tf.tile(tf.reshape(ats,[nmol,natom3,1,4]),[1,1,nelep,1])
	ps = tf.tile(tf.reshape(pinds,[1,1,nelep,1]),[nmol,natom3,1,1])
	ToMask = tf.concat([ats,ps],axis=3)
	GoodInds = tf.boolean_mask(ToMask,Mask)
	nnz = tf.shape(GoodInds)[0]
	# Good Inds has shape << nmol * natom3 * nelep X 5 (mol, i,j,k,l=element pair.)
	# and contains all the indices we actually want to compute, Now we just slice, gather and compute. 
	mijs = tf.slice(GoodInds,[0,0],[nnz,3])
	miks = tf.concat([tf.slice(GoodInds,[0,0],[nnz,2]),tf.slice(GoodInds,[0,3],[nnz,1])],axis=-1)
	Rij = DifferenceVectorsSet(R,prec) # nmol X atom X atom X 3
	A = tf.gather_nd(Rij,mijs)
	B = tf.gather_nd(Rij,miks)
	RijRik = tf.reduce_sum(A*B,axis=1)
	RijRij = tf.sqrt(tf.reduce_sum(A*A,axis=1)+infinitesimal)
	RikRik = tf.sqrt(tf.reduce_sum(B*B,axis=1)+infinitesimal)
	denom = RijRij*RikRik
	# Mask any troublesome entries.
	ToACos = RijRik/denom
	ToACos = tf.where(tf.greater_equal(ToACos,1.0),tf.ones_like(ToACos, dtype=prec),ToACos)
	ToACos = tf.where(tf.less_equal(ToACos,-1.0),-1.0*tf.ones_like(ToACos, dtype=prec),ToACos)
	Thetaijk = tf.acos(ToACos)
	zetatmp = tf.cast(tf.reshape(SFPs_[0],[1,nzeta,neta,ntheta,nr]),prec)
	thetatmp = tf.cast(tf.tile(tf.reshape(SFPs_[2],[1,nzeta,neta,ntheta,nr]),[nnz,1,1,1,1]),prec)
	# Broadcast the thetas and ToCos together
	tct = tf.tile(tf.reshape(Thetaijk,[nnz,1,1,1,1]),[1,nzeta,neta,ntheta,nr])
	ToCos = tct-thetatmp
	Tijk = tf.cos(ToCos) # shape: natom3 X ...
	# complete factor 1 
	fac1 = tf.pow(tf.cast(2.0, prec),1.0-zetatmp)*tf.pow((1.0+Tijk),zetatmp)
	etmp = tf.cast(tf.reshape(SFPs_[1],[1,nzeta,neta,ntheta,nr]),prec) # ijk X zeta X eta ....
	rtmp = tf.cast(tf.reshape(SFPs_[3],[1,nzeta,neta,ntheta,nr]),prec) # ijk X zeta X eta ....
	ToExp = ((RijRij+RikRik)/2.0)
	tet = tf.tile(tf.reshape(ToExp,[nnz,1,1,1,1]),[1,nzeta,neta,ntheta,nr]) - rtmp
	fac2 = tf.exp(-etmp*tet*tet)
	# And finally the last two factors
	fac3 = tf.where(tf.greater_equal(RijRij,R_cut),tf.zeros_like(RijRij, dtype=prec),0.5*(tf.cos(3.14159265359*RijRij/R_cut)+1.0))
	fac4 = tf.where(tf.greater_equal(RikRik,R_cut),tf.zeros_like(RikRik, dtype=prec),0.5*(tf.cos(3.14159265359*RikRik/R_cut)+1.0))
	# assemble the full symmetry function for all triples.
	fac34t =  tf.tile(tf.reshape(fac3*fac4,[nnz,1,1,1,1]),[1,nzeta,neta,ntheta,nr])
	Gm = tf.reshape(fac1*fac2*fac34t,[nnz*nzeta*neta*ntheta*nr]) # nnz X nzeta X neta X ntheta X nr
	# Finally scatter out the symmetry functions where they belong. 
	mil_jk = tf.concat([tf.slice(GoodInds,[0,0],[nnz,2]),tf.slice(GoodInds,[0,4],[nnz,1]),tf.reshape(tf.reduce_prod(tf.slice(GoodInds,[0,2],[nnz,2]),axis=1),[nnz,1])],axis=-1)
	mil_jk_Outer = tf.tile(tf.reshape(mil_jk,[nnz,1,4]),[1,nsym,1])
	# So the above is Mol, i, l... now must outer nzeta,neta,ntheta,nr to finish the indices. 
	p1 = tf.tile(tf.reshape(tf.range(nzeta),[nzeta,1]),[1,neta])
	p2 = tf.tile(tf.reshape(tf.concat([p1,tf.tile(tf.reshape(tf.range(neta),[1,neta]),[nzeta,1])],axis=-1),[nzeta,neta,1,2]),[1,1,ntheta,1])
	p3 = tf.tile(tf.reshape(tf.concat([p2,tf.tile(tf.reshape(tf.range(ntheta),[1,1,ntheta,1]),[nzeta,neta,1,1])],axis=-1),[nzeta,neta,ntheta,1,3]),[1,1,1,nr,1])
	p4 = tf.reshape(tf.concat([p3,tf.tile(tf.reshape(tf.range(nr),[1,1,1,nr,1]),[nzeta,neta,ntheta,1,1])],axis=-1),[1,nzeta,neta,ntheta,nr,4])
	p5 = tf.reshape(tf.reduce_prod(p4,axis=-1),[1,nsym,1]) # scatter_nd only supports upto rank 5... so gotta smush this... 
	p6 = tf.tile(p5,[nnz,1,1]) # should be nnz X nsym 
	ind = tf.reshape(tf.concat([mil_jk_Outer,p6],axis=-1),[nnz*nsym,5]) # This is now nnz*nzeta*neta*ntheta*nr X 8 -  m,i,l,jk,zeta,eta,theta,r
	to_reduce = tf.scatter_nd(ind,Gm,[nmol,natom,nelep,natom2,nsym])
	return tf.reduce_sum(to_reduce,axis=3)

In [182]:
import numpy as np
Pi = 3.1415
xyz = tf.Variable([[[0.,0.,0.],[1.0,0.,0.],[0.,0.,5.],[1.,1.,5.],[0,0,0],[0,0,0],[0,0,0]],[[0.,0.,0.],[1.0,0.,0.],[0.,0.,5.],[1.,1.,5.],[0,0,0],[0,0,0],[0,0,0]]],trainable=False)
Z = tf.Variable([[1,1,7,8,0,0,0],[1,1,7,8,0,0,0]],trainable=False)
eleps = tf.Variable([[1,1],[1,7],[1,8],[7,8],[7,7]])
zetas = tf.Variable([[8.0]])
etas = tf.Variable([[4.0]])
AN1_num_a_As = 8
thetas = tf.Variable([ 2.0*Pi*i/AN1_num_a_As for i in range (0, AN1_num_a_As)])
AN1_num_r_Rs = 22
AN1_r_Rc = 10.6
AN1_a_Rc = 10.1
rs = tf.Variable([ AN1_r_Rc*i/AN1_num_r_Rs for i in range (0, AN1_num_r_Rs)])
rcut = AN1_r_Rc

# Create a parameter tensor. 4 x nzeta X neta X ntheta X nr 
p1 = tf.tile(tf.reshape(zetas,[1,1,1,1,1]),[1,1,AN1_num_a_As,AN1_num_r_Rs,1])
p2 = tf.tile(tf.reshape(etas,[1,1,1,1,1]),[1,1,AN1_num_a_As,AN1_num_r_Rs,1])
p3 = tf.tile(tf.reshape(thetas,[1,1,AN1_num_a_As,1,1]),[1,1,1,AN1_num_r_Rs,1])
p4 = tf.tile(tf.reshape(rs,[1,1,1,AN1_num_r_Rs,1]),[1,1,AN1_num_a_As,1,1])
SFP = tf.concat([p1,p2,p3,p4],axis=4)
SFP = tf.transpose(SFP, perm=[4,0,1,2,3])

# A test of the above.
init = tf.global_variables_initializer()
with tf.Session() as session:
    session.run(init)
    SFs = session.run(TFSymASet(xyz,Z,eleps,SFP,rcut))
    print SFs.shape, SFs.mean(), SFs
    print SFs[0,0], SFs[0,0].mean(), SFs[0,0].max()
    print SFs[0,1], SFs[0,1].mean(), SFs[0,1].max()    

(2, 7, 5, 176) 0.0275127456473 [[[[  5.53030842   0.           0.         ...,   0.           0.           0.        ]
   [  3.07828003   0.           0.         ...,   0.           0.           0.        ]
   [  2.91364522   0.           0.         ...,   0.           0.           0.        ]
   [  1.62129348   0.           0.         ...,   0.           0.           0.        ]
   [  1.71288813   0.           0.         ...,   0.           0.           0.        ]]

  [[  5.53030842   0.           0.         ...,   0.           0.           0.        ]
   [  2.99498118   0.           0.         ...,   0.           0.           0.        ]
   [  2.99544992   0.           0.         ...,   0.           0.           0.        ]
   [  1.62172937   0.           0.         ...,   0.           0.           0.        ]
   [  1.62203415   0.           0.         ...,   0.           0.           0.        ]]

  [[  6.66820493   0.           0.         ...,   0.           0.           0.       

In [197]:
def TFDistance(A):
	"""
	Compute a distance matrix of A, a coordinate matrix
	Using the factorization:
	Dij = <i|i> - 2<i|j> + <j,j>
	Args:
		A: a Nx3 matrix
	Returns:
		D: a NxN matrix
	"""
	r = tf.reduce_sum(A*A, 1)
	r = tf.reshape(r, [-1, 1]) # For the later broadcast.
	# Tensorflow can only reverse mode grad the sqrt if all these elements
	# are nonzero
	D = r - 2*tf.matmul(A, tf.transpose(A)) + tf.transpose(r) + 1e-26
	return tf.sqrt(D)

def TFDistances(r_):
	"""
	Returns distance matrices batched over mols
	Args:
		r_: Nmol X MaxNAtom X 3 coordinate tensor
	Returns
		D: Nmol X MaxNAtom X MaxNAtom Distance tensor.
	"""
	rm = tf.einsum('ijk,ijk->ij',r_,r_) # Mols , Rsq.
	rshp = tf.shape(rm)
	rmt = tf.tile(rm, [1,rshp[1]])
	rmt = tf.reshape(rmt,[rshp[0],rshp[1],rshp[1]])
	rmtt = tf.transpose(rmt,perm=[0,2,1])
	# Tensorflow can only reverse mode grad of sqrt if all these elements
	# are nonzero
	D = rmt - 2*tf.einsum('ijk,ilk->ijl',r_,r_) + rmtt + 1e-26
	return tf.sqrt(D)

def BumpEnergy(h,w,xyz,x):
	"""
	A potential energy which is just the sum of gaussians
	with height h and width w at positions xyz sampled at x.
	This uses distance matrices to maintain rotational invariance.

	Args:
		h: bump height
		w: bump width
		xyz: a nbump X N X 3 tensor of bump centers.
		x: (n X 3) tensor representing the point at which the energy is sampled.
	"""
	bshp = tf.shape(xyz)
	nbump = bshp[0]
	xshp = tf.shape(x)
	nx = xshp[0]
	Ds = TFDistances(xyz) # nbump X MaxNAtom X MaxNAtom Distance tensor.
	Dx = TFDistance(x) # MaxNAtom X MaxNAtom Distance tensor.

	sqrt2pi = tf.constant(2.50662827463100,dtype = tf.float64)
	w2 = w*w
	infinitesimal = tf.constant(0.000000000000000000000000001,dtype = tf.float64)
	# here I should tf.assert bshp[1] == nx but fuggit.
	rij = xyz - tf.tile(tf.reshape(x,[1,nx,nx]),[nbump,1,1])
	Ds = tf.einsum('ijk,ijk->i',rij,rij)
	#exps = (h/(w*sqrt2pi))*tf.exp(-0.5*Ds/w2) if you wanna normalize.
	exps = h*tf.exp(-0.5*Ds/w2)
	return -1.0*tf.reduce_sum(exps,axis=0)

def BumpForce(BumpHeight,BumpWidth,BumpCoords,x_):
	h = tf.Variable(BumpHeight,dtype = tf.float64)
	w = tf.Variable(BumpWidth,dtype = tf.float64)
	xyzs = tf.Variable(BumpCoords,dtype = tf.float64)
	x = tf.Variable(x_,dtype = tf.float64)
	init = tf.global_variables_initializer()
	import sys
	sys.stderr = sys.stdout
	with tf.Session() as session:
		session.run(init)
		return session.run(tf.gradients(BumpEnergy(h,w,xyzs,x),x))

In [198]:
BumpForce(0.1,1.0,np.array([np.eye(3),2*np.eye(3)]),np.array([[0.1,0.2,0.3],[0.,0.1,0.2],[0.,0.,0.3]]))

[array([[-0.02989667,  0.00651353,  0.0097703 ],
        [ 0.        , -0.02989667,  0.00651353],
        [ 0.        ,  0.        , -0.02338314]])]

In [ ]:
i = tf.constant(0)
c = lambda i: tf.less(i, 10)
b = lambda i: tf.add(i, 1)
r = tf.while_loop(c, b, [i])

In [ ]:
init = tf.global_variables_initializer()
with tf.Session() as session:
    session.run(init)
    print session.run(r)

In [ ]:
def TFRoll(t,n):
    """
    Rotate a 1-tensor around it's last axis by n
    Left. 
    
    Args: 
        t: a tensor
        n: number of rolls. 
    """
    return tf.concat([t[n:],t[:n]],0)

In [ ]:
t = tf.Variable([0,1,2,3,4,5,6,7,8,9])
init = tf.global_variables_initializer()
with tf.Session() as session:
    session.run(init)
    print session.run(TFRoll(t,2))

In [ ]:
i = tf.constant(0)
t = tf.Variable([0,1,2,3,4,5,6,7,8,9])
c = lambda x,y: tf.less(x, 10)
b = lambda x,y: [tf.add(x, 1), TFRoll(y,x)]
r = b(i,t)
init = tf.global_variables_initializer()
with tf.Session() as session:
    session.run(init)
    print session.run(r)

In [ ]:
def TFRoll(t):
    """
    Rotate a 2-tensor around it's last axis by 1
    Left. 
    
    Args: 
        t: a tensor
        n: number of rolls. 
    """
    return tf.concat([t[:,n:],t[:,:n]],1)
def NextRoll(x,y): 
    """
    Rolls one more in the matrix y, and updates y. 
    """
    y=TFRoll(y,x)
    y.set_shape(10)
    return (tf.add(x, 1),y)
r = tf.while_loop(c, NextRoll, loop_vars=[i,t], shape_invariants=[tf.TensorShape(()), tf.TensorShape([10])])
init = tf.global_variables_initializer()
with tf.Session() as session:
    session.run(init)
    print session.run(r)

In [ ]:
elems = (tf.Variable([1, 2, 3]), tf.Variable([-1, 1, -1]))
alternate = tf.map_fn(lambda x: x[0] * x[1], elems, dtype=tf.int32)
init = tf.global_variables_initializer()
with tf.Session() as session:
    session.run(init)
    print session.run(alternate)

In [ ]:
t = tf.Variable([0,1,2,3,4,5,6,7,8,9])
r = tf.map_fn(lambda x: tf.add(x[0],x[1]) ,(t,t),dtype=tf.int32)
init = tf.global_variables_initializer()
with tf.Session() as session:
    session.run(init)
    print session.run(r)

In [ ]:
def TFDistances(r_):
	"""
	Returns distance matrices batched over mols
    It's counterintuitive but this uses a foil-ed out dot product. 
    
	Args:
		r_: Nmol X MaxNAtom X 3 coordinate tensor
	Returns
		D: Nmol X MaxNAtom X MaxNAtom Distance tensor.
	"""
	rm = tf.einsum('ijk,ijk->ij',r_,r_) # Mols , Rsq.
	rshp = tf.shape(rm)
	rmt = tf.tile(rm, [1,rshp[1]])
	rmt = tf.reshape(rmt,[rshp[0],rshp[1],rshp[1]])
	rmtt = tf.transpose(rmt,perm=[0,2,1])
	# Tensorflow can only reverse mode grad of sqrt if all these elements
	# are nonzero
	D = rmt - 2*tf.einsum('ijk,ilk->ijl',r_,r_) + rmtt + 0.000000000000000000000000001
	return tf.sqrt(D)



In [ ]:
def DifferenceVectors(r_):
    """
    Given a maxnatom X 3 tensor of coordinates this
    returns a maxnatom X maxnatom X 3 tensor of Rij 
    """
    natom = tf.shape(r_)[0]
    ri = tf.tile(tf.reshape(r_,[1,natom,3]),[natom,1,1])
    rj = tf.transpose(ri,perm=(1,0,2))
    return (ri-rj)

def DifferenceVectorsSet(r_):
    """
    Given a maxnatom X 3 tensor of coordinates this
    returns a maxnatom X maxnatom X 3 tensor of Rij 
    """
    natom = tf.shape(r_)[1]
    nmol = tf.shape(r_)[0]
    ri = tf.tile(tf.reshape(r_,[nmol,1,natom,3]),[1,natom,1,1])
    rj = tf.transpose(ri,perm=(0,2,1,3))
    return (ri-rj)

def Zouter(Z):
    """
    Returns the outer product of atomic numbers for one molecule. 

    Args:
        Z: MaxNAtom X 1 Z tensor
    Returns
        Z1Z2: MaxNAtom X MaxNAtom X 2 Z1Z2 tensor.
    """
    zshp = tf.shape(Z)
    Zs = tf.reshape(Z,[zshp[0],1])
    z1 = tf.tile(Zs, [1,zshp[0]])
    z2 = tf.transpose(z1,perm=[1,0])
    return tf.transpose(tf.stack([z1,z2],axis=0),perm=[1,2,0])


def ZouterSet(Z):
    """
    Returns the outer product of atomic numbers for one molecule. 

    Args:
        Z: MaxNAtom X 1 Z tensor
    Returns
        Z1Z2: MaxNAtom X MaxNAtom X 2 Z1Z2 tensor.
    """
    zshp = tf.shape(Z)
    Zs = tf.reshape(Z,[zshp[0],zshp[1],1])
    z1 = tf.tile(Zs, [1,1,zshp[1]])
    z2 = tf.transpose(z1,perm=[0,2,1])
    return tf.transpose(tf.stack([z1,z2],axis=1),perm=[0,2,3,1])

In [ ]:
t = tf.Variable([0,1,2,3,4,5,6,7,8,9])
t2 = tf.Variable([[1,6,7,7],[1,6,8,8]])
r = tf.Variable([[[0,1,2],[3,4,5]],[[0,1,2],[2,3,4]]])
init = tf.global_variables_initializer()
with tf.Session() as session:
    session.run(init)
#    print session.run(TFRoll(t,1))
    print session.run(ZouterSet(t2))

In [ ]:
# Test the distance matrices. 
xyz_ = tf.Variable([[[0.,0.,0.],[10.0,0.,0.],[0.,0.,5.]],[[0.,0.,2.],[0.,1.,9.],[0.,1.,20.]]],trainable=False)
init = tf.global_variables_initializer()
with tf.Session() as session:
    session.run(init)
    print session.run(TFDistances(xyz_))

In [ ]:
# A dummy CZ and set of elements to generate the descriptor. 
# Also dummy 
Cz = tf.Variable([[[1.,0.,0.,0.],[1.,10.0,0.,0.],[8.,0.,0.,5.],[0.,0.,0.,0.]],[[7,0.,0.,2.],[1,0.,1.,9.],[8,0.,1.,20.],[7,0.,1.,-20.]]],trainable=False)
Eles = tf.Variable([[1,1],[1,7],[1,8],[7,8]],trainable=False)
"""zetas_: List of zeta angular series parameters.
		etas_: list of eta radial parameters.
		thetas_: list of theta angular parameters.
		Rs_: list of Rs_ radial parameters."""
Zetas = tf.Variable([8],trainable=False)
Etas = tf.Variable([4],trainable=False)
dAngle = 2.*3.1415
Thetas = tf.Variable(tf.range(0.0,2.*3.1415,dAngle),trainable=False)
rRs = tf.Variable(tf.range(0.01,4.6,.46),trainable=False)
aRs = tf.Variable(tf.range(0.01,4.6,.46),trainable=False)
rRc = tf.Variable(4.6,trainable=False)# Cutoffs. 
aRc = tf.Variable(4.6,trainable=False)

In [ ]:
# This bit of code shows you how to get indexing tensors which match some pairing of elements. 
elemps = tf.Variable([[1,1],[1,7],[1,8],[7,8]])
Z1Z2 = tf.Variable([[[1,7],[1,8],[1,8]],[[7,8],[8,8],[8,8]],[[7,8],[8,8],[8,8]]]) # maxnatom X maxnatom X 2
ToSelect = tf.reduce_all(tf.equal(tf.reshape(elemps,[4,1,1,2]),tf.reshape(Z1Z2,[1,3,3,2])),axis=-1)
shp = tf.shape(ToSelect)
init = tf.global_variables_initializer()
with tf.Session() as session:
    session.run(init)
    print session.run(tf.shape(ToSelect))
    res1,s1 =  session.run([ToSelect,shp])
    print s1   
    print res1[1]
    print res1[0]  
    print res1[3]  

In [ ]:
def AllTriples(natom = 4):
    """Returns all possible triples of integers between zero and natom. 
    
    Args: 
        natom: max integer
    Returns: 
        A natom X natom X natom X 3 tensor of all triples. 
    """
    rng=tf.range(natom)
    v1 = tf.tile(tf.reshape(rng,[natom,1]),[1,natom])
    v2 = tf.tile(tf.reshape(rng,[1,natom]),[natom,1])
    v3 = tf.transpose(tf.stack([v1,v2],0),perm=[1,2,0])
    # V3 is now all pairs (nat x nat x 2). now do the same with another to make nat X 3
    v4 = tf.tile(tf.reshape(v3,[natom,natom,1,2]),[1,1,natom,1])
    v5 = tf.tile(tf.reshape(rng,[1,1,natom,1]),[natom,natom,1,1])
    v6 = tf.concat([v4,v5], axis = 3) # All triples in the range. 
    return v6

def AllTriplesSet(rng):
    """Returns all possible triples of integers between zero and natom. 
    
    Args: 
        natom: max integer
    Returns: 
        A natom X natom X natom X 3 tensor of all triples. 
    """
    natom = tf.shape(rng)[1]
    nmol = tf.shape(rng)[0]
    v1 = tf.tile(tf.reshape(rng,[nmol,natom,1]),[1,1,natom])
    v2 = tf.tile(tf.reshape(rng,[nmol,1,natom]),[1,natom,1])
    v3 = tf.transpose(tf.stack([v1,v2],1),perm=[0,2,3,1])
    # V3 is now all pairs (nat x nat x 2). now do the same with another to make nat X 3
    v4 = tf.tile(tf.reshape(v3,[nmol,natom,natom,1,2]),[1,1,1,natom,1])
    v5 = tf.tile(tf.reshape(rng,[nmol,1,1,natom,1]),[1,natom,natom,1,1])
    v6 = tf.concat([v4,v5], axis = 4) # All triples in the range. 
    v7 = tf.tile(tf.reshape(tf.range(nmol),[nmol,1,1,1,1]),[1,natom,natom,natom,1])
    v8 = tf.concat([v7,v6], axis = -1)
    return v8

def Zouter(Z):
    """
    Returns the outer product of atomic numbers for one molecule. 

    Args:
        Z: MaxNAtom X 1 Z tensor
    Returns
        Z1Z2: MaxNAtom X MaxNAtom X 2 Z1Z2 tensor.
    """
    zshp = tf.shape(Z)
    Zs = tf.reshape(Z,[zshp[0],1])
    z1 = tf.tile(Zs, [1,zshp[0]])
    z2 = tf.transpose(z1,perm=[1,0])
    return tf.transpose(tf.stack([z1,z2],axis=0),perm=[1,2,0])

In [ ]:
# A test of the above. 
with tf.Session() as session:
    Z = tf.Variable([1,1,7,8],trainable=False)
    natom = 4
    init = tf.global_variables_initializer()
    session.run(init)
    print session.run(AllTriples(4))
    print session.run(Zouter(Z))

In [ ]:
def DifferenceVectors(r_):
    """
    Given a maxnatom X 3 tensor of coordinates this
    returns a maxnatom X maxnatom X 3 tensor of Rij 
    """
    natom = tf.shape(r_)[0]
    ri = tf.tile(tf.reshape(r_,[1,natom,3]),[natom,1,1])
    rj = tf.transpose(ri,perm=(1,0,2))
    return (ri-rj)

def DifferenceVectorsSet(r_):
    """
    Given a nmol X maxnatom X 3 tensor of coordinates this
    returns a nmol X maxnatom X maxnatom X 3 tensor of Rij 
    """
    natom = tf.shape(r_)[1]
    nmol = tf.shape(r_)[0]
    ri = tf.tile(tf.reshape(r_,[nmol,1,natom,3]),[1,natom,1,1])
    rj = tf.transpose(ri,perm=(0,2,1,3))
    return (ri-rj)

def AllTriples(rng):
    """Returns all possible triples of an input list.
    
    Args: 
        rng: a 1-D tensor. 
    Returns: 
        A natom X natom X natom X 3 tensor of all triples of entries from rng. 
    """
    rshp = tf.shape(rng)
    natom = rshp[0]
    v1 = tf.tile(tf.reshape(rng,[natom,1]),[1,natom])
    v2 = tf.tile(tf.reshape(rng,[1,natom]),[natom,1])
    v3 = tf.transpose(tf.stack([v1,v2],0),perm=[1,2,0])
    # V3 is now all pairs (nat x nat x 2). now do the same with another to make nat X 3
    v4 = tf.tile(tf.reshape(v3,[natom,natom,1,2]),[1,1,natom,1])
    v5 = tf.tile(tf.reshape(rng,[1,1,natom,1]),[natom,natom,1,1])
    v6 = tf.concat([v4,v5], axis = 3) # All triples in the range. 
    return v6


def AllTriplesSet(rng):
    """Returns all possible triples of integers between zero and natom. 
    
    Args: 
        natom: max integer
    Returns: 
        A Nmol X natom X natom X natom X 4 tensor of all triples. 
    """
    natom = tf.shape(rng)[1]
    nmol = tf.shape(rng)[0]
    v1 = tf.tile(tf.reshape(rng,[nmol,natom,1]),[1,1,natom])
    v2 = tf.tile(tf.reshape(rng,[nmol,1,natom]),[1,natom,1])
    v3 = tf.transpose(tf.stack([v1,v2],1),perm=[0,2,3,1])
    # V3 is now all pairs (nat x nat x 2). now do the same with another to make nat X 3
    v4 = tf.tile(tf.reshape(v3,[nmol,natom,natom,1,2]),[1,1,1,natom,1])
    v5 = tf.tile(tf.reshape(rng,[nmol,1,1,natom,1]),[1,natom,natom,1,1])
    v6 = tf.concat([v4,v5], axis = 4) # All triples in the range. 
    v7 = tf.tile(tf.reshape(tf.range(nmol),[nmol,1,1,1,1]),[1,natom,natom,natom,1])
    v8 = tf.concat([v7,v6], axis = -1)
    return v8


def Zouter(Z):
    """
    Returns the outer product of atomic numbers for one molecule. 

    Args:
        Z: MaxNAtom X 1 Z tensor
    Returns
        Z1Z2: MaxNAtom X MaxNAtom X 2 Z1Z2 tensor.
    """
    zshp = tf.shape(Z)
    Zs = tf.reshape(Z,[zshp[0],1])
    z1 = tf.tile(Zs, [1,zshp[0]])
    z2 = tf.transpose(z1,perm=[1,0])
    return tf.transpose(tf.stack([z1,z2],axis=0),perm=[1,2,0])

def ZouterSet(Z):
    """
    Returns the outer product of atomic numbers for one molecule. 

    Args:
        Z: MaxNAtom X 1 Z tensor
    Returns
        Z1Z2: MaxNAtom X MaxNAtom X 2 Z1Z2 tensor.
    """
    zshp = tf.shape(Z)
    Zs = tf.reshape(Z,[zshp[0],zshp[1],1])
    z1 = tf.tile(Zs, [1,1,zshp[1]])
    z2 = tf.transpose(z1,perm=[0,2,1])
    return tf.transpose(tf.stack([z1,z2],axis=1),perm=[0,2,3,1])

def jacobian(y, x):
    y_flat = tf.reshape(y, (-1,))
    jacobian_flat = tf.stack([tf.gradients(y_i, x)[0] for y_i in tf.unstack(y_flat)])
    return tf.reshape(jacobian_flat, y.shape.concatenate(x.shape))

def TFSymR(R, Zs, elems, RSFPs, R_cut):
    """
    A tensorflow implementation of the radial AN1 symmetry function for a single input molecule. 

    Args:
        R: a maxnatom X 3 tensor of coordinates. 
        Zs : maxnatom X 1 tensor of atomic numbers.  
        elems_: a tensor of elements present in the data. 
        RSFPs: A symmetry function parameter tensor 2 X neta X nRs. 
        For example, RSFPs[0,0,0] is the first eta parameter. RSFPs[1,1] is the second R parameter.  
        R_cut: Radial Cutoff

    Returns:
        Digested Mol. In the shape maxnatom X nelem X nEta X nRs
    """
    inp_shp = tf.shape(R)
    natom = inp_shp[0]
    natom2 = natom*natom
    natom3 = natom*natom2
    nele = tf.shape(elems)[0]
    pshape = tf.shape(RSFPs)
    neta = pshape[0]
    nr = pshape[1]

    # atom triples. 
    ats = AllTriples(tf.range(natom))
    # Z triples. Used to scatter element contribs. 
    ZTrips = AllTriples(Zs)
    
    # Construct the angle triples acos(<Rij,Rik>/|Rij||Rik|) and mask them onto the correct output
    # Get Rij, Rik... 
    Ri_inds = tf.slice(ats,[0,0,0,0],[natom,natom,natom,1])
    Rj_inds = tf.slice(ats,[0,0,0,1],[natom,natom,natom,1])
    Rk_inds = tf.slice(ats,[0,0,0,2],[natom,natom,natom,1])
    Rij_inds = tf.reshape(tf.concat([Ri_inds,Rj_inds],axis=3),[natom3,2])
    Rik_inds = tf.reshape(tf.concat([Ri_inds,Rk_inds],axis=3),[natom3,2])
    Rij = DifferenceVectors(R) # atom X atom X 3 
    # Pull out the appropriate pairs of distances from Rij. 
    A = tf.gather_nd(Rij,Rij_inds)
    B = tf.gather_nd(Rij,Rik_inds)
    RijRik = tf.einsum("ij,ij->i",A,B)
    RijRij = tf.sqrt(tf.einsum("ij,ij->i",A,A)+infinitesimal)
    RikRik = tf.sqrt(tf.einsum("ij,ij->i",B,B)+infinitesimal)
    denom = RijRij*RikRik
    # Mask any troublesome entries. 
    ToACos = RijRik/denom
    ToACos = tf.where(tf.greater_equal(ToACos,1.0),tf.ones_like(ToACos),ToACos)
    ToACos = tf.where(tf.less_equal(ToACos,-1.0),-1.0*tf.ones_like(ToACos),ToACos) 
    Thetaijk = tf.acos(ToACos)
    # Finally construct the thetas for all the triples. 
    zetatmp = tf.reshape(SFPs_[0],[1,nzeta,neta,ntheta,nr])
    thetatmp = tf.reshape(SFPs_[2],[1,nzeta,neta,ntheta,nr])
    # Broadcast the thetas and ToCos together 
    tct = tf.reshape(Thetaijk,[natom3,1,1,1,1])
    Tijk = tf.cos(tct-thetatmp) # shape: natom3 X ... 
    # complete factor 1 for all j,k
    fac1 = tf.pow(2.0,1.0-zetatmp)*tf.pow((1.0+Tijk),zetatmp)
    # Construct Rij + Rik/2  for all jk!=i 
    # make the etas,R's broadcastable onto this and vice versa. 
    etmp = tf.reshape(SFPs_[1],[1,nzeta,neta,ntheta,nr]) # ijk X zeta X eta .... 
    rtmp = tf.reshape(SFPs_[3],[1,nzeta,neta,ntheta,nr]) # ijk X zeta X eta ....     
    ToExp = ((RijRij+RikRik)/2.0)
    tet = tf.reshape(ToExp,[natom3,1,1,1,1]) - rtmp
    fac2 = tf.exp(-etmp*tet*tet)
    # And finally the last two factors
    fac3 = tf.where(tf.greater_equal(RijRij,R_cut),tf.zeros_like(RijRij),0.5*(tf.cos(3.14159265359*RijRij/R_cut)+1.0))
    fac4 = tf.where(tf.greater_equal(RikRik,R_cut),tf.zeros_like(RikRik),0.5*(tf.cos(3.14159265359*RikRik/R_cut)+1.0))
    # Zero out the diagonal contributions (i==j or i==k)
    mask1 = tf.reshape(tf.where(tf.equal(Ri_inds,Rj_inds),tf.zeros_like(Ri_inds,dtype=tf.float32),tf.ones_like(Ri_inds,dtype=tf.float32)),[natom3])
    mask2 = tf.reshape(tf.where(tf.equal(Ri_inds,Rk_inds),tf.zeros_like(Ri_inds,dtype=tf.float32),tf.ones_like(Ri_inds,dtype=tf.float32)),[natom3])  
    fac34t =  tf.reshape(fac3*fac4*mask1*mask2,[natom3,1,1,1,1])
    # assemble the full symmetry function for all triples. 
    # Use broadcasting to mask these out... 
    Gm = fac1*fac2*fac34t # Gm so far has shape atom3 X nzeta X neta X ntheta X nr
    # Now, finally Scatter the element contributions and sum over jk. 
    Rjk_inds = tf.reshape(tf.concat([Rj_inds,Rk_inds],axis=3),[natom3,2])
    Z1Z2 = Zouter(Z)
    ZPairs = tf.gather_nd(Z1Z2,Rjk_inds) # should have shape natom3 X 2
    # Create a tensor which selects out components where jk = elep[i]
    # This is done by broadcasting our natom X natom X natom X zeta... tensor. 
    # onto a tensor which has an added dimension for the element pairs, and reducing over jk.
    ElemReduceMask = tf.reduce_all(tf.equal(tf.reshape(ZPairs,[natom3,1,2]),tf.reshape(eleps_,[1,nelep,2])),axis=-1)
    # So this tensor has dim natom3 X nelep we broadcast over it's shape and reduce_sum. 
    GmToMask = tf.tile(tf.reshape(Gm,[natom,natom2,1,nzeta,neta,ntheta,nr]),[1,1,nelep,1,1,1,1])
    ERMask = tf.tile(tf.reshape(ElemReduceMask,[natom,natom2,nelep,1,1,1,1]),[1,1,1,nzeta,neta,ntheta,nr])
    ToRS = tf.where(ERMask,GmToMask,tf.zeros_like(GmToMask))
    GMA = tf.reduce_sum(ToRS,axis=[1])
    return GMA

def TFSymA(R, Zs, eleps_, SFPs_, R_cut):
    """
    A tensorflow implementation of the angular AN1 symmetry function for a single input molecule. 
    Here j,k are all other atoms, but implicitly the output
    is separated across elements as well. eleps_ is a list of element pairs
    G = 2**(1-zeta) \sum_{j,k \neq i} (Angular triple) (radial triple) f_c(R_{ij}) f_c(R_{ik})
    a-la MolEmb.cpp. Also depends on PARAMS for zeta, eta, theta_s r_s

    Args:
        R: a maxnatom X 3 tensor of coordinates. 
        Zs : maxnatom X 1 tensor of atomic numbers.  
        eleps_: a nelepairs X 2 tensor of element pairs present in the data. 
        SFP: A symmetry function parameter tensor having the number of elements 
        as the SF output. 4 X nzeta X neta X thetas X nRs. For example, SFPs_[0,0,0,0,0] 
        is the first zeta parameter. SFPs_[3,0,0,0,1] is the second R parameter.  
        R_cut: Radial Cutoff

    Returns:
        Digested Mol. In the shape maxnatom X nelepairs X nZeta X nEta X nThetas X nRs
    """
    inp_shp = tf.shape(R)
    natom = inp_shp[0]
    natom2 = natom*natom
    natom3 = natom*natom2
    nelep = tf.shape(eleps_)[0]
    pshape = tf.shape(SFPs_)
    nzeta = pshape[1]
    neta = pshape[2]
    ntheta = pshape[3]
    nr = pshape[4]
    nsym = nzeta*neta*ntheta*nr
    infinitesimal = 0.000000000000000000000000001

    # atom triples. 
    ats = AllTriples(tf.range(natom))
    # Z triples. Used to scatter element contribs. 
    #ZTrips = AllTriples(Zs)
    
    # Construct the angle triples acos(<Rij,Rik>/|Rij||Rik|) and mask them onto the correct output
    # Get Rij, Rik... 
    Ri_inds = tf.slice(ats,[0,0,0,0],[natom,natom,natom,1])
    Rj_inds = tf.slice(ats,[0,0,0,1],[natom,natom,natom,1])
    Rk_inds = tf.slice(ats,[0,0,0,2],[natom,natom,natom,1])
    Rij_inds = tf.reshape(tf.concat([Ri_inds,Rj_inds],axis=3),[natom3,2])
    Rik_inds = tf.reshape(tf.concat([Ri_inds,Rk_inds],axis=3),[natom3,2])
    Rij = DifferenceVectors(R) # atom X atom X 3 
    # Pull out the appropriate pairs of distances from Rij. 
    A = tf.gather_nd(Rij,Rij_inds)
    B = tf.gather_nd(Rij,Rik_inds)
    RijRik = tf.einsum("ij,ij->i",A,B)
    RijRij = tf.sqrt(tf.einsum("ij,ij->i",A,A)+infinitesimal)
    RikRik = tf.sqrt(tf.einsum("ij,ij->i",B,B)+infinitesimal)
    denom = RijRij*RikRik
    # Mask any troublesome entries. 
    ToACos = RijRik/denom
    ToACos = tf.where(tf.greater_equal(ToACos,1.0),tf.ones_like(ToACos),ToACos)
    ToACos = tf.where(tf.less_equal(ToACos,-1.0),-1.0*tf.ones_like(ToACos),ToACos) 
    Thetaijk = tf.acos(ToACos)
    # Finally construct the thetas for all the triples. 
    zetatmp = tf.reshape(SFPs_[0],[1,nzeta,neta,ntheta,nr])
    thetatmp = tf.reshape(SFPs_[2],[1,nzeta,neta,ntheta,nr])
    # Broadcast the thetas and ToCos together 
    tct = tf.reshape(Thetaijk,[natom3,1,1,1,1])
    Tijk = tf.cos(tct-thetatmp) # shape: natom3 X ... 
    # complete factor 1 for all j,k
    fac1 = tf.pow(2.0,1.0-zetatmp)*tf.pow((1.0+Tijk),zetatmp)
    # Construct Rij + Rik/2  for all jk!=i 
    # make the etas,R's broadcastable onto this and vice versa. 
    etmp = tf.reshape(SFPs_[1],[1,nzeta,neta,ntheta,nr]) # ijk X zeta X eta .... 
    rtmp = tf.reshape(SFPs_[3],[1,nzeta,neta,ntheta,nr]) # ijk X zeta X eta ....     
    ToExp = ((RijRij+RikRik)/2.0)
    tet = tf.reshape(ToExp,[natom3,1,1,1,1]) - rtmp
    fac2 = tf.exp(-etmp*tet*tet)
    # And finally the last two factors
    fac3 = tf.where(tf.greater_equal(RijRij,R_cut),tf.zeros_like(RijRij),0.5*(tf.cos(3.14159265359*RijRij/R_cut)+1.0))
    fac4 = tf.where(tf.greater_equal(RikRik,R_cut),tf.zeros_like(RikRik),0.5*(tf.cos(3.14159265359*RikRik/R_cut)+1.0))
    # Zero out the diagonal contributions (i==j or i==k)
    mask1 = tf.reshape(tf.where(tf.equal(Ri_inds,Rj_inds),tf.zeros_like(Ri_inds,dtype=tf.float32),tf.ones_like(Ri_inds,dtype=tf.float32)),[natom3])
    mask2 = tf.reshape(tf.where(tf.equal(Ri_inds,Rk_inds),tf.zeros_like(Ri_inds,dtype=tf.float32),tf.ones_like(Ri_inds,dtype=tf.float32)),[natom3])
    # Also mask out the lower triangle. (j>k)
    # mask3 = tf.reshape(tf.where(tf.greater(Rj_inds,Rk_inds),tf.zeros_like(Ri_inds,dtype=tf.float32),tf.ones_like(Ri_inds,dtype=tf.float32)),[natom3])    
    # assemble the full symmetry function for all triples. 
    fac34t =  tf.reshape(fac3*fac4*mask1*mask2,[natom3,1,1,1,1])
    # Use broadcasting to mask these out... 
    Gm = fac1*fac2*fac34t # Gm so far has shape atom3 X nzeta X neta X ntheta X nr
    # Now, finally Scatter the element contributions and sum over jk. 
    Rjk_inds = tf.reshape(tf.concat([Rj_inds,Rk_inds],axis=3),[natom3,2])
    Z1Z2 = Zouter(Z)
    ZPairs = tf.gather_nd(Z1Z2,Rjk_inds) # should have shape natom3 X 2
    # Create a tensor which selects out components where jk = elep[i]
    # This is done by broadcasting our natom X natom X natom X zeta... tensor. 
    # onto a tensor which has an added dimension for the element pairs, and reducing over jk.
    ElemReduceMask = tf.reduce_all(tf.equal(tf.reshape(ZPairs,[natom3,1,2]),tf.reshape(eleps_,[1,nelep,2])),axis=-1)
    # So this tensor has dim natom3 X nelep we broadcast over it's shape and reduce_sum. 
    GmToMask = tf.tile(tf.reshape(Gm,[natom,natom2,1,nzeta,neta,ntheta,nr]),[1,1,nelep,1,1,1,1])
    ERMask = tf.tile(tf.reshape(ElemReduceMask,[natom,natom2,nelep,1,1,1,1]),[1,1,1,nzeta,neta,ntheta,nr])
    ToRS = tf.where(ERMask,GmToMask,tf.zeros_like(GmToMask))
    GMA = tf.reduce_sum(ToRS,axis=[1])
    return GMA

def TFSymASet(R, Zs, eleps_, SFPs_, R_cut):
    """
    A tensorflow implementation of the angular AN1 symmetry function for a single input molecule. 
    Here j,k are all other atoms, but implicitly the output
    is separated across elements as well. eleps_ is a list of element pairs
    G = 2**(1-zeta) \sum_{j,k \neq i} (Angular triple) (radial triple) f_c(R_{ij}) f_c(R_{ik})
    a-la MolEmb.cpp. Also depends on PARAMS for zeta, eta, theta_s r_s

    Args:
        R: a nmol X maxnatom X 3 tensor of coordinates. 
        Zs : nmol X maxnatom X 1 tensor of atomic numbers.  
        eleps_: a nelepairs X 2 tensor of element pairs present in the data. 
        SFP: A symmetry function parameter tensor having the number of elements 
        as the SF output. 4 X nzeta X neta X thetas X nRs. For example, SFPs_[0,0,0,0,0] 
        is the first zeta parameter. SFPs_[3,0,0,0,1] is the second R parameter.  
        R_cut: Radial Cutoff

    Returns:
        Digested Mol. In the shape maxnatom X nelepairs X nZeta X nEta X nThetas X nRs
    """
    inp_shp = tf.shape(R)
    nmol = inp_shp[0]
    natom = inp_shp[1]
    natom2 = natom*natom
    natom3 = natom*natom2
    nelep = tf.shape(eleps_)[0]
    pshape = tf.shape(SFPs_)
    nzeta = pshape[1]
    neta = pshape[2]
    ntheta = pshape[3]
    nr = pshape[4]
    nsym = nzeta*neta*ntheta*nr
    infinitesimal = 0.000000000000000000000000001

    # atom triples. 
    ats = AllTriplesSet(tf.tile(tf.reshape(tf.range(natom),[1,natom]),[nmol,1]))
    # Z triples. Used to scatter element contribs. 
    #ZTrips = AllTriplesSet(Zs)
    
    # Construct the angle triples acos(<Rij,Rik>/|Rij||Rik|) and mask them onto the correct output
    # Get Rij, Rik... 
    Rm_inds = tf.slice(ats,[0,0,0,0,0],[nmol,natom,natom,natom,1])
    Ri_inds = tf.slice(ats,[0,0,0,0,1],[nmol,natom,natom,natom,1])
    Rj_inds = tf.slice(ats,[0,0,0,0,2],[nmol,natom,natom,natom,1])
    Rk_inds = tf.slice(ats,[0,0,0,0,3],[nmol,natom,natom,natom,1])
    Rij_inds = tf.reshape(tf.concat([Rm_inds,Ri_inds,Rj_inds],axis=4),[nmol,natom3,3])
    Rik_inds = tf.reshape(tf.concat([Rm_inds,Ri_inds,Rk_inds],axis=4),[nmol,natom3,3])
    Rij = DifferenceVectorsSet(R) # nmol X atom X atom X 3 
    # Pull out the appropriate pairs of distances from Rij. 
    A = tf.gather_nd(Rij,Rij_inds)
    B = tf.gather_nd(Rij,Rik_inds)
    RijRik = tf.einsum("ijk,ijk->ij",A,B)
    RijRij = tf.sqrt(tf.einsum("ijk,ijk->ij",A,A)+infinitesimal)
    RikRik = tf.sqrt(tf.einsum("ijk,ijk->ij",B,B)+infinitesimal)
    denom = RijRij*RikRik
    # Mask any troublesome entries. 
    ToACos = RijRik/denom
    ToACos = tf.where(tf.greater_equal(ToACos,1.0),tf.ones_like(ToACos),ToACos)
    ToACos = tf.where(tf.less_equal(ToACos,-1.0),-1.0*tf.ones_like(ToACos),ToACos) 
    Thetaijk = tf.acos(ToACos)
    # Finally construct the thetas for all the triples. 
    zetatmp = tf.reshape(SFPs_[0],[1,nzeta,neta,ntheta,nr])
    thetatmp = tf.reshape(SFPs_[2],[1,nzeta,neta,ntheta,nr])
    # Broadcast the thetas and ToCos together 
    tct = tf.reshape(Thetaijk,[nmol,natom3,1,1,1,1])
    Tijk = tf.cos(tct-thetatmp) # shape: natom3 X ... 
    # complete factor 1 for all j,k
    fac1 = tf.pow(2.0,1.0-zetatmp)*tf.pow((1.0+Tijk),zetatmp)
    # Construct Rij + Rik/2  for all jk!=i 
    # make the etas,R's broadcastable onto this and vice versa. 
    etmp = tf.reshape(SFPs_[1],[1,nzeta,neta,ntheta,nr]) # ijk X zeta X eta .... 
    rtmp = tf.reshape(SFPs_[3],[1,nzeta,neta,ntheta,nr]) # ijk X zeta X eta ....     
    ToExp = ((RijRij+RikRik)/2.0)
    tet = tf.reshape(ToExp,[nmol,natom3,1,1,1,1]) - rtmp
    fac2 = tf.exp(-etmp*tet*tet)
    # And finally the last two factors
    fac3 = tf.where(tf.greater_equal(RijRij,R_cut),tf.zeros_like(RijRij),0.5*(tf.cos(3.14159265359*RijRij/R_cut)+1.0))
    fac4 = tf.where(tf.greater_equal(RikRik,R_cut),tf.zeros_like(RikRik),0.5*(tf.cos(3.14159265359*RikRik/R_cut)+1.0))
    # Zero out the diagonal contributions (i==j or i==k)
    mask1 = tf.reshape(tf.where(tf.equal(Ri_inds,Rj_inds),tf.zeros_like(Ri_inds,dtype=tf.float32),tf.ones_like(Ri_inds,dtype=tf.float32)),[nmol,natom3])
    mask2 = tf.reshape(tf.where(tf.equal(Ri_inds,Rk_inds),tf.zeros_like(Ri_inds,dtype=tf.float32),tf.ones_like(Ri_inds,dtype=tf.float32)),[nmol,natom3])
    # Also mask out the lower triangle. (j>k)
    # mask3 = tf.reshape(tf.where(tf.greater(Rj_inds,Rk_inds),tf.zeros_like(Ri_inds,dtype=tf.float32),tf.ones_like(Ri_inds,dtype=tf.float32)),[natom3])    
    # assemble the full symmetry function for all triples. 
    fac34t =  tf.reshape(fac3*fac4*mask1*mask2,[nmol,natom3,1,1,1,1])
    # Use broadcasting to mask these out... 
    Gm = fac1*fac2*fac34t # Gm so far has shape atom3 X nzeta X neta X ntheta X nr
    # Now, finally Scatter the element contributions and sum over jk. 
    Rjk_inds = tf.reshape(tf.concat([Rm_inds,Rj_inds,Rk_inds],axis=4),[nmol,natom3,3])
    Z1Z2 = ZouterSet(Z)
    ZPairs = tf.gather_nd(Z1Z2,Rij_inds) # should have shape natom3 X 2
    # Create a tensor which selects out components where jk = elep[i]
    # This is done by broadcasting our natom X natom X natom X zeta... tensor. 
    # onto a tensor which has an added dimension for the element pairs, and reducing over jk.
    ElemReduceMask = tf.reduce_all(tf.equal(tf.reshape(ZPairs,[nmol,natom3,1,2]),tf.reshape(eleps_,[1,1,nelep,2])),axis=-1)
    # So this tensor has dim natom3 X nelep we broadcast over it's shape and reduce_sum. 
    Gmtmp = tf.reshape(Gm,[nmol*natom*natom2,1,nzeta,neta,ntheta,nr])
    GmToMasktmp = tf.tile(Gmtmp,[1,nelep,1,1,1,1])
    GmToMask = tf.reshape(GmToMasktmp,[nmol,natom,natom2,nelep,nzeta,neta,ntheta,nr])
    #GmToMask = tf.tile(tf.reshape(Gm,[nmol,natom,natom2,1,nzeta,neta,ntheta,nr]),[1,1,1,nelep,1,1,1,1])
    ElemReduceMasktmp = tf.reshape(ElemReduceMask,[nmol*natom*natom2*nelep,1,1,1,1])
    ERMasktmp = tf.tile(ElemReduceMasktmp,[1,nzeta,neta,ntheta,nr])
    ERMask = tf.reshape(ERMasktmp,[nmol,natom,natom2,nelep,nzeta,neta,ntheta,nr])
    #ERMask = tf.tile(tf.reshape(ElemReduceMask,[nmol,natom,natom2,nelep,1,1,1,1]),[1,1,1,1,nzeta,neta,ntheta,nr])
    ToRS = tf.where(ERMask,GmToMask,tf.zeros_like(GmToMask))
    GMA = tf.reduce_sum(ToRS,axis=[2])
    return GMA,thetatmp,tct

In [ ]:
tf.__version__

In [ ]:
import numpy as np
Pi = 3.1415
xyz = tf.Variable([[0.,0.,0.],[1.0,0.,0.],[0.,0.,5.],[1.,1.,5.],[0,0,0],[0,0,0],[0,0,0]],trainable=False)
Z = tf.Variable([1,1,7,8,0,0,0],trainable=False)
eleps = tf.Variable([[1,1],[1,7],[1,8],[7,8],[7,7]])
zetas = tf.Variable([[8.0]])
etas = tf.Variable([[4.0]])
AN1_num_a_As = 8
thetas = tf.Variable([ 2.0*Pi*i/AN1_num_a_As for i in range (0, AN1_num_a_As)])
AN1_num_r_Rs = 22
AN1_r_Rc = 4.6
AN1_a_Rc = 3.1
rs = tf.Variable([ AN1_r_Rc*i/AN1_num_r_Rs for i in range (0, AN1_num_r_Rs)])
rcut = AN1_r_Rc

# Create a parameter tensor. 4 x nzeta X neta X ntheta X nr 
p1 = tf.tile(tf.reshape(zetas,[1,1,1,1,1]),[1,1,AN1_num_a_As,AN1_num_r_Rs,1])
p2 = tf.tile(tf.reshape(etas,[1,1,1,1,1]),[1,1,AN1_num_a_As,AN1_num_r_Rs,1])
p3 = tf.tile(tf.reshape(thetas,[1,1,AN1_num_a_As,1,1]),[1,1,1,AN1_num_r_Rs,1])
p4 = tf.tile(tf.reshape(rs,[1,1,1,AN1_num_r_Rs,1]),[1,1,AN1_num_a_As,1,1])
SFP = tf.concat([p1,p2,p3,p4],axis=4)
SFP = tf.transpose(SFP, perm=[4,0,1,2,3])

# A test of the above.
init = tf.global_variables_initializer()
with tf.Session() as session:
    session.run(init)
#    print session.run(SFP)
#    SFs = session.run(TFSymA(xyz,Z,eleps,SFP,rcut))
    SymA = TFSymA(xyz,Z,eleps,SFP,rcut)
    SymA.set_shape([7, 5, 1, 1, 8, 22])
    print SymA.shape
    print "grad:"
    grad = session.run(jacobian(SymA,xyz))
    print grad, grad.shape

In [ ]:
import numpy as np
Pi = 3.1415
xyz = tf.Variable([[0.,0.,0.],[1.0,0.,0.],[0.,0.,5.],[1.,1.,5.]],trainable=False)
Z = tf.Variable([1,1,7,8],trainable=False)
eleps = tf.Variable([[1,1],[1,7],[1,8],[7,8],[7,7]])
zetas = tf.Variable([[8.0]])
etas = tf.Variable([[4.0]])
AN1_num_a_As = 8
thetas = tf.Variable([ 2.0*Pi*i/AN1_num_a_As for i in range (0, AN1_num_a_As)])
AN1_num_r_Rs = 22
AN1_r_Rc = 4.6
AN1_a_Rc = 3.1
rs = tf.Variable([ AN1_r_Rc*i/AN1_num_r_Rs for i in range (0, AN1_num_r_Rs)])
rcut = AN1_r_Rc

# Create a parameter tensor. 4 x nzeta X neta X ntheta X nr 
p1 = tf.tile(tf.reshape(zetas,[1,1,1,1,1]),[1,1,AN1_num_a_As,AN1_num_r_Rs,1])
p2 = tf.tile(tf.reshape(etas,[1,1,1,1,1]),[1,1,AN1_num_a_As,AN1_num_r_Rs,1])
p3 = tf.tile(tf.reshape(thetas,[1,1,AN1_num_a_As,1,1]),[1,1,1,AN1_num_r_Rs,1])
p4 = tf.tile(tf.reshape(rs,[1,1,1,AN1_num_r_Rs,1]),[1,1,AN1_num_a_As,1,1])
SFP = tf.concat([p1,p2,p3,p4],axis=4)
SFP = tf.transpose(SFP, perm=[4,0,1,2,3])

# A test of the above.
init = tf.global_variables_initializer()
with tf.Session() as session:
    session.run(init)
#    print session.run(SFP)
    SFs,thetatmp,Gm = session.run(TFSymA(xyz,Z,eleps,SFP,rcut))
    print SFs.shape, SFs.mean(), SFs.max(), thetatmp.shape, Gm.shape
    print SFs

In [ ]:
import numpy as np
Pi = 3.1415
xyz = tf.Variable([[[0.,0.,0.],[1.0,0.,0.],[0.,0.,5.],[1.,1.,5.],[0,0,0],[0,0,0],[0,0,0]],[[0.,0.,0.],[1.0,0.,0.],[0.,0.,5.],[1.,1.,5.],[0,0,0],[0,0,0],[0,0,0]]],trainable=False)
Z = tf.Variable([[1,1,7,8,0,0,0],[1,1,7,8,0,0,0]],trainable=False)
eleps = tf.Variable([[1,1],[1,7],[1,8],[7,8],[7,7]])
zetas = tf.Variable([[8.0]])
etas = tf.Variable([[4.0]])
AN1_num_a_As = 8
thetas = tf.Variable([ 2.0*Pi*i/AN1_num_a_As for i in range (0, AN1_num_a_As)])
AN1_num_r_Rs = 22
AN1_r_Rc = 4.6
AN1_a_Rc = 3.1
rs = tf.Variable([ AN1_r_Rc*i/AN1_num_r_Rs for i in range (0, AN1_num_r_Rs)])
rcut = AN1_r_Rc

# Create a parameter tensor. 4 x nzeta X neta X ntheta X nr 
p1 = tf.tile(tf.reshape(zetas,[1,1,1,1,1]),[1,1,AN1_num_a_As,AN1_num_r_Rs,1])
p2 = tf.tile(tf.reshape(etas,[1,1,1,1,1]),[1,1,AN1_num_a_As,AN1_num_r_Rs,1])
p3 = tf.tile(tf.reshape(thetas,[1,1,AN1_num_a_As,1,1]),[1,1,1,AN1_num_r_Rs,1])
p4 = tf.tile(tf.reshape(rs,[1,1,1,AN1_num_r_Rs,1]),[1,1,AN1_num_a_As,1,1])
SFP = tf.concat([p1,p2,p3,p4],axis=4)
SFP = tf.transpose(SFP, perm=[4,0,1,2,3])

# A test of the above.
init = tf.global_variables_initializer()
with tf.Session() as session:
    session.run(init)
#    print session.run(SFP)
    SFs,thetatmp,Gm = session.run(TFSymASet(xyz,Z,eleps,SFP,rcut))
    print SFs.shape, SFs.mean(), SFs.max(), thetatmp.shape, Gm.shape
    print Gm